<a href="https://colab.research.google.com/github/jpo85/meshroom_colab/blob/main/MeshroomColab_Ver1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. configura il runtime in modalità GPU e fai il check**

abilitare le GPU per il notebook:

Vai a Modifica → Impostazioni blocco note
Seleziona GPU dal menu a discesa Acceleratore hardware
Successivamente, confermeremo che possiamo connetterci alla GPU con tensorflow

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

**1. connetti a Google Drive**

connetti a google drive utilizzando l'authorization code


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

**2. crea le folder necessarie e fa il download di meshroom:**
*   /content - contiene la root del pacchetto meshroom che sarà eseguito
*   /content/temp - contiene il pacchetto meshroom scaricato
*   /content/meshroom - contiene tutti gli eseguibili di meshroom scompattati e pronti per essere eseguiti








In [ ]:
%cd /content
!mkdir temp
!mkdir meshroom
%cd temp
!wget -N https://github.com/alicevision/meshroom/releases/download/v2019.2.0/Meshroom-2019.2.0-linux.tar.gz
!tar -xvf Meshroom-2019.2.0-linux.tar.gz -C ../meshroom


**3. esegui meshroom**
*   il nodi salvano temporaneamente nella folder **/tmp/MeshroomCache**
*   il file obj finale viene salvato nella folder **/content/drive/output**


In [ ]:
!/content/meshroom/Meshroom-2019.2.0/meshroom_photogrammetry --input '/content/drive/MyDrive/input' --output '/content/drive/MyDrive/output'

**4. Preview Mesh usando Trimesh** 

ancora in fase sperimentale

In [ ]:
!pip install numpy
!pip install trimesh
%cd /content/drive/output

lancia preview

In [ ]:
import numpy as np
import trimesh
mesh = trimesh.load_mesh('texturedMesh.obj')
mesh.show()